使用我个人的账户试试

# Exercise 2: Creating Redshift Cluster using the AWS python SDK 
## An example of Infrastructure-as-code

In [1]:
import pandas as pd
import boto3
import json

## STEP 0: (Prerequisite) Save the AWS Access key

### 1. Create a new IAM user
IAM service is a global service, meaning newly created IAM users are not restricted to a specific region by default.
- Go to [AWS IAM service](https://console.aws.amazon.com/iam/home#/users) and click on the "**Add user**" button to create a new IAM user in your AWS account. 
- Choose a name of your choice. 
- Select "*Programmatic access*" as the access type. Click Next. 
- Choose the *Attach existing policies directly* tab, and select the "**AdministratorAccess**". Click Next. 
- Skip adding any tags. Click Next. 
- Review and create the user. It will show you a pair of access key ID and secret.
- Take note of the pair of access key ID and secret. This pair is collectively known as **Access key**. 

<center>
<img style="float: center;height:300px;" src="images/AWS_IAM_1.png"><br><br>
Snapshot of a pair of an Access key
</center>

### <font color='red'>2. Save the access key and secret</font>
Edit the file `dwh.cfg` in the same folder as this notebook and save the access key and secret against the following variables:
```bash
KEY= <YOUR_AWS_KEY>
SECRET= <YOUR_AWS_SECRET>
```
    
For example:
```bash
KEY=6JW3ATLQ34PH3AKI
SECRET=wnoBHA+qUBFgwCRHJqgqrLU0i
```

### 3. Troubleshoot
If your keys are not working, such as getting an `InvalidAccessKeyId` error, then you cannot retrieve them again. You have either of the following two options:

1. **Option 1 - Create a new pair of access keys for the existing user**

 - Go to the [IAM dashboard](https://console.aws.amazon.com/iam/home) and view the details of the existing (Admin) user. 

 - Select on the **Security credentials** tab, and click the **Create access key** button. It will generate a new pair of access key ID and secret.

 - Save the new access key ID and secret in your `dwh.cfg` file


<center>
<img style="float: center;height:450px;" src="images/AWS_IAM_2.png"><br><br>
Snapshot of creating a new Access keys for the existing user
</center>

2. **Option 2 - Create a new IAM user with Admin access** - Refer to the instructions at the top. 

# Load DWH Params from a file

In [2]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh-4.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,8
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,dwhClusterTry4extra
4,DWH_DB,dwh
5,DWH_DB_USER,dwhuser
6,DWH_DB_PASSWORD,Passw0rd
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,dwhRole


## Create clients and resources for IAM, EC2, S3, and Redshift

To interact with EC2 and S3, utilize `boto3.resource`; for IAM and Redshift, use `boto3.client`. If you require additional details on boto3, refer to the [boto3 documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html).

**Note**: We create clients and resources in the **us-west-2** region. Choose the same region in your AWS Web Console to see these resources.

In [3]:
import boto3

ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

# Check out the sample data sources on S3

In [4]:
sampleDbBucket =  s3.Bucket("awssampledbuswest2")
for obj in sampleDbBucket.objects.filter(Prefix="ssbgz"):
    print(obj)


s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/customer0002_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/dwdate.tbl.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0000_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0001_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0002_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0003_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0004_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0005_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0006_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0007_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='s

In [5]:
for obj in sampleDbBucket.objects.all():
    print(obj)

s3.ObjectSummary(bucket_name='awssampledbuswest2', key='load/')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='load/customer-fw-manifest')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='load/customer-fw.tbl-000')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='load/customer-fw.tbl-000.bak')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='load/customer-fw.tbl-001')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='load/customer-fw.tbl-002')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='load/customer-fw.tbl-003')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='load/customer-fw.tbl-004')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='load/customer-fw.tbl-005')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='load/customer-fw.tbl-006')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='load/customer-fw.tbl-007')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='load/customer-fw.tbl.log')
s3.ObjectSummary(b

# STEP 1: IAM ROLE
- Create an IAM Role that makes Redshift able to access S3 bucket (ReadOnly)

`dwhRole = iam.create_role(...)`
- 这段代码非常重要，它在AWS中设置了Redshift所需的IAM角色权限
- 这部分代码创建了一个IAM角色，让Redshift可以代表你访问其他AWS服务
- 这很重要因为：没有这个角色，Redshift将无法读取S3中的数据


`iam.attach_role_policy(...)`
- 给角色添加了AmazonS3ReadOnlyAccess策略
- 这让Redshift集群能够读取S3存储桶中的数据
- 这很重要因为：如果没有这个权限，你的Redshift将无法从S3加载数据进行分析

`roleArn = iam.get_role(...)['Role']['Arn']`
- 获取角色的Amazon资源名称（ARN）,这个ARN在创建Redshift集群时需要用到
- ARN (Amazon Resource Name) 是AWS中用来唯一标识资源的一个标识符
- 身份证号码 - 用来唯一标识一个人
- 手机号码 - 用来唯一联系一个人
- ARN - 用来唯一标识AWS中的一个资源。如果不提供，就像给你一个手机，但不告诉你SIM卡号，你就无法打电话一样

In [6]:
from botocore.exceptions import ClientError

#1.1 Create the role, 
try:
    print("1.1 Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    
    
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

1.1 Creating a new IAM Role
An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name dwhRole already exists.
1.2 Attaching Policy
1.3 Get the IAM role ARN
arn:aws:iam::339713039693:role/dwhRole


# STEP 2:  Redshift Cluster

- Create a [RedShift Cluster](https://console.aws.amazon.com/redshiftv2/home)
- For complete arguments to `create_cluster`, see [docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift.html#Redshift.Client.create_cluster)

In [7]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn],

        #Public Access

        PubliclyAccessible=True
    )
except Exception as e:
    print(e)

## 2.1 *Describe* the cluster to see its status
- run this block several times until the cluster status becomes `Available`

In [10]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', None)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwhclustertry4extra
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,dwhuser
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhclustertry4extra.czgl7wspitsl.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-08df3248a3b917e72
7,NumberOfNodes,8


<h2> 2.2 Take note of the cluster <font color='red'> endpoint and role ARN </font> </h2>

<font color='red'>DO NOT RUN THIS unless the cluster status becomes "Available". Make ure you are checking your Amazon Redshift cluster in the **us-west-2** region. </font>

这段代码是从Redshift集群属性中提取两个重要的连接信息：
- DWH_ENDPOINT：这是Redshift集群的访问地址，用于后续连接数据库。就像是数据仓库的"门牌号码"
- DWH_ROLE_ARN：这是之前我们创建的IAM角色的ARN，用于验证权限设置是否正确

In [11]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)  # 用于检查的目的


DWH_ENDPOINT ::  dwhclustertry4extra.czgl7wspitsl.us-west-2.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::339713039693:role/dwhRole


这里arn和之前redshift的arn am.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn'] 是一回事吗

是的，这是同一个ARN！让我解释一下这两处代码获取ARN的不同方式：

1. 创建角色时获取ARN：
```python
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
```
- 这是从IAM服务直接获取刚创建的角色的ARN
- 这是在创建Redshift集群之前获取的

2. 从Redshift集群属性中获取ARN：
```python
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
```
- 这是从已经运行的Redshift集群的属性中获取同一个ARN
- 这是确认Redshift集群确实在使用我们之前创建的那个角色

这就像是：
1. 第一次是："我创建了一个钥匙，它的编号是xxx"
2. 第二次是："让我检查一下我的仓库用的是不是编号xxx的钥匙"

这两个值应该是完全相同的，只是获取的途径不同：
- 一个是从"钥匙管理处"（IAM）获取
- 一个是从"仓库"（Redshift）查看使用的钥匙编号

这样做的目的是为了验证权限配置是否正确，确保Redshift确实在使用我们指定的IAM角色。

## STEP 3: Open an incoming  TCP port to access the cluster ednpoint

In [15]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

    

ec2.SecurityGroup(id='sg-089332983f4a97320')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


这段代码是在配置AWS安全组(Security Group)，让外部可以访问Redshift集群。让我详细解释：

1. 获取VPC和安全组：
```python
vpc = ec2.Vpc(id=myClusterProps['VpcId'])  # 获取VPC
defaultSg = list(vpc.security_groups.all())[0]  # 获取默认安全组
```
- 从Redshift集群属性中获取VPC ID
- 找到这个VPC的默认安全组

2. 配置入站规则：
```python
defaultSg.authorize_ingress(
    GroupName=defaultSg.group_name,
    CidrIp='0.0.0.0/0',        # 允许所有IP地址访问
    IpProtocol='TCP',          # 使用TCP协议
    FromPort=int(DWH_PORT),    # 开始端口(5439)
    ToPort=int(DWH_PORT)       # 结束端口(5439)
)
```

这就像是配置防火墙规则：
- `CidrIp='0.0.0.0/0'` - 允许任何IP地址访问（⚠️要小心使用）
- `FromPort=ToPort=5439` - 只开放Redshift默认端口

安全提示：
- `0.0.0.0/0` 意味着对所有IP开放，在生产环境中应该限制为特定IP范围
- 比如公司办公网络的IP范围: `'192.168.1.0/24'`

这段代码的目的是：
1. 让你能从本地电脑连接到Redshift
2. 让其他工具（如BI工具）能连接到Redshift
3. 但也要注意安全性，生产环境应该更严格地限制访问

# STEP 4: Make sure you can connect to the cluster

In [16]:
import psycopg2

try:
    conn = psycopg2.connect(
        dbname=DWH_DB,
        user=DWH_DB_USER,
        password=DWH_DB_PASSWORD,
        host=DWH_ENDPOINT,
        port=DWH_PORT
    )
    
    cur = conn.cursor()
    
    # 测试查询
    cur.execute("SELECT VERSION()")
    version = cur.fetchone()
    print(version)
    
except Exception as e:
    print("Error:", e)
finally:
    if 'cur' in locals():
        cur.close()
    if 'conn' in locals():
        conn.close()
        

('PostgreSQL 8.0.2 on i686-pc-linux-gnu, compiled by GCC gcc (GCC) 3.4.2 20041017 (Red Hat 3.4.2-6.fc3), Redshift 1.0.107351',)


先暂停，进入下一个lab

# STEP 5: Clean up your resources

<b><font color='red'>DO NOT RUN THIS UNLESS YOU ARE SURE <br/> 
    We will be using these resources in the next exercises</span></b>

In [7]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

{'Cluster': {'ClusterIdentifier': 'dwhcluster',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'ClusterAvailabilityStatus': 'Modifying',
  'MasterUsername': 'dwhuser',
  'DBName': 'dwh',
  'Endpoint': {'Address': 'dwhcluster.czgl7wspitsl.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2025, 2, 16, 4, 42, 18, 205000, tzinfo=tzutc()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ManualSnapshotRetentionPeriod': -1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-089332983f4a97320',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-2.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-08df3248a3b917e72',
  'AvailabilityZone': 'us-west-2b',
  'PreferredMaintenanceWindow': 'sun:10:30-sun:11:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 4,
  'P

- run this block several times until the cluster really deleted

In [8]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

NameError: name 'prettyRedshiftProps' is not defined

In [9]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
#### CAREFUL!!

{'ResponseMetadata': {'RequestId': '276a033d-f7cc-4018-83a8-d48cea37a98e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 16 Feb 2025 06:52:12 GMT',
   'x-amzn-requestid': '276a033d-f7cc-4018-83a8-d48cea37a98e',
   'content-type': 'text/xml',
   'content-length': '200'},
  'RetryAttempts': 0}}

In [10]:
import psycopg2

try:
    conn = psycopg2.connect(
        host="dwhcluster.cjlk0pfgllo5.us-west-2.redshift.amazonaws.com",
        dbname="dwh",
        user="dwhuser",
        password="Passw0rd",
        port=5439
    )
    print("Connected successfully!")
except Exception as e:
    print("Connection failed:", e)

Connection failed: could not translate host name "dwhcluster.cjlk0pfgllo5.us-west-2.redshift.amazonaws.com" to address: nodename nor servname provided, or not known



In [11]:
# 打印集群状态
print(redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]['ClusterStatus'])

# 确认endpoint
print(redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]['Endpoint'])

deleting
{'Address': 'dwhcluster.czgl7wspitsl.us-west-2.redshift.amazonaws.com', 'Port': 5439}


In [ ]:
# 检查当前安全组规则
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print("Current Security Group:", defaultSg.id)
    
    # 重新设置入站规则
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',  # 允许所有IP访问
        IpProtocol='TCP',
        FromPort=5439,
        ToPort=5439
    )
except Exception as e:
    print("Security Group Error:", e)

Current Security Group: sg-089b857887dc10194
Security Group Error: An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


In [12]:
# 检查集群状态
response = redshift.describe_clusters(
    ClusterIdentifier=DWH_CLUSTER_IDENTIFIER
)
cluster_status = response['Clusters'][0]['ClusterStatus']
print("Cluster Status:", cluster_status)

ClusterNotFoundFault: An error occurred (ClusterNotFound) when calling the DescribeClusters operation: Cluster dwhcluster not found.

In [29]:
# 打印VPC和子网信息
print("VPC ID:", myClusterProps['VpcId'])
print("Subnet ID:", myClusterProps['Subnet'])

VPC ID: vpc-0ea59bc9b4003dc29


KeyError: 'Subnet'

In [30]:
# 获取完整的集群信息
response = redshift.describe_clusters(
    ClusterIdentifier=DWH_CLUSTER_IDENTIFIER
)
cluster_info = response['Clusters'][0]

# 打印关键信息
print("Endpoint:", cluster_info['Endpoint'])
print("VPC Security Groups:", cluster_info['VpcSecurityGroups'])
print("Availability Zone:", cluster_info['AvailabilityZone'])
print("Publicly Accessible:", cluster_info['PubliclyAccessible'])

Endpoint: {'Address': 'dwhcluster.cjlk0pfgllo5.us-west-2.redshift.amazonaws.com', 'Port': 5439}
VPC Security Groups: [{'VpcSecurityGroupId': 'sg-089b857887dc10194', 'Status': 'active'}]
Availability Zone: us-west-2a
Publicly Accessible: False


In [31]:
try:
    response = redshift.modify_cluster(
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        PubliclyAccessible=True
    )
    print("Modified cluster to be publicly accessible")
except Exception as e:
    print(e)

Modified cluster to be publicly accessible
